## L4 - 3: Uncertainty Estimation Model

### Instructions
- Given the Swiss heart disease dataset we have been working with, create an uncertainty estimation model that accounts for Epistemic Uncertainty as well. Provide the mean and standard deviation outputs.
- https://blog.tensorflow.org/2019/03/regression-with-probabilistic-layers-in.html

### Use Exercise 1 Preprocessing Code

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
#from https://archive.ics.uci.edu/ml/datasets/Heart+Disease
swiss_dataset_path = "./data/heart_disease_data/processed_swiss.csv"
swiss_df = pd.read_csv(swiss_dataset_path).replace('?', np.nan)

In [2]:
column_list = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'oldpeak', 'slope', 'ca', 'thal', 'num_label']

In [3]:
cleveland_df = pd.read_csv("./data/heart_disease_data/processed.cleveland.txt",  names=column_list).replace('?', np.nan)

In [4]:
combined_heart_df = pd.concat([swiss_df, cleveland_df])

In [5]:
selected_features = ['sex',  'age', 'trestbps', 'thalach' ]
bp_df = combined_heart_df[selected_features].replace({1:"male", 0:"female"})

In [10]:
#for simplicity will drop rows with null since predictor is null 
clean_df = bp_df.dropna()

In [12]:
clean_df['trestbps'] = clean_df['trestbps'].astype(float)
clean_df['thalach'] = clean_df['thalach'].astype(float)

/Users/md/opt/anaconda3/envs/test_udacity_hc_env/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/md/opt/anaconda3/envs/test_udacity_hc_env/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [13]:
#adapted from https://www.tensorflow.org/tutorials/structured_data/feature_columns
def df_to_dataset(df, predictor,  batch_size=32):
    df = df.copy()
    labels = df.pop(predictor)
    ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
    ds = ds.shuffle(buffer_size=len(df))
    ds = ds.batch(batch_size)
    return ds

In [14]:
#split 80 20 train test split - not ideal 
train_dataset = clean_df.sample(frac=0.8,random_state=0)
test_dataset = clean_df.drop(train_dataset.index)

In [15]:
PREDICTOR_FIELD = 'trestbps'
batch_size = 128
train_ds = df_to_dataset(train_dataset, PREDICTOR_FIELD, batch_size=batch_size)
test_ds = df_to_dataset(test_dataset, PREDICTOR_FIELD, batch_size=batch_size)

In [17]:
# create TF numeric feature
tf_numeric_age_feature = tf.feature_column.numeric_column(key='age', default_value=0, dtype=tf.float64)

In [18]:
b_list = [ 0, 18, 25, 40, 55, 65, 80, 100]
#create TF bucket feature from numeric feature
tf_numeric_age_feature
tf_bucket_age_feature = tf.feature_column.bucketized_column(source_column=tf_numeric_age_feature, boundaries= b_list)

In [19]:
#using list b/c small number of unique values
gender_vocab = tf.feature_column.categorical_column_with_vocabulary_list(
      'sex', bp_df['sex'].unique())
gender_one_hot = tf.feature_column.indicator_column(gender_vocab)

In [20]:
# add cross features - use example from TF
crossed_age_gender_feature = tf.feature_column.crossed_column([tf_bucket_age_feature, gender_vocab], hash_bucket_size=1000)
tf_crossed_age_gender_feature = tf.feature_column.indicator_column(crossed_age_gender_feature)

In [21]:
feature_columns = [ tf_crossed_age_gender_feature, tf_bucket_age_feature, gender_one_hot ]
dense_feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

### Build Model

In [52]:
import tensorflow_probability as tfp

In [54]:
'''
Adapted from Tensorflow Probability Regression tutorial  https://github.com/tensorflow/probability/blob/master/tensorflow_probability/examples/jupyter_notebooks/Probabilistic_Layers_Regression.ipynb    
'''
def posterior_mean_field(kernel_size, bias_size=0, dtype=None):
    n = kernel_size + bias_size
    c = np.log(np.expm1(1.))
    return tf.keras.Sequential([
        tfp.layers.VariableLayer(2*n, dtype=dtype),
        tfp.layers.DistributionLambda(lambda t: tfp.distributions.Independent(
            tfp.distributions.Normal(loc=t[..., :n],
                                     scale=1e-5 + tf.nn.softplus(c + t[..., n:])),
            reinterpreted_batch_ndims=1)),
    ])


def prior_trainable(kernel_size, bias_size=0, dtype=None):
    n = kernel_size + bias_size
    return tf.keras.Sequential([
        tfp.layers.VariableLayer(n, dtype=dtype),
        tfp.layers.DistributionLambda(lambda t: tfp.distributions.Independent(
            tfp.distributions.Normal(loc=t, scale=1),
            reinterpreted_batch_ndims=1)),
    ])
